# Traveling Salesman Problem (TSP)

In [1]:
using Pkg
Pkg.activate(".")

using LinearAlgebra
using Printf
using Random

using Gurobi
using JuMP

using Plots

┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1317
┌ Warning: Module JSON with build ID 479443834401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Warning: Module JSON with build ID 479443834401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Warning: Module JSON with build ID 479443834401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Warning: Module JSON with build ID 479443834401 is missing from the cache.
│ This may mean JSON [682c06a0-de6a-54ab-a142-c8b1cf79cde6] does not support precompilation but is imported by a module that does.
└

## Problem formulation

We have $N$ cities to visit.
Each city must be visited exactly once, and we must return to city $1$ at the end of the trip.

Mathematical formulation:
\begin{align}
    \min_{x} \quad
      & \sum_{i, j} D_{i, j} x_{i, j}\\
      \text{s.t.} \quad
      & x_{i, i} = 0, 
          && \forall i \in \mathcal{N},\\
      & \sum_{j \in \mathcal{N}} x_{i, j} = 1, 
          && \forall i \in \mathcal{N},\\
      & \sum_{i \in \mathcal{N}} x_{i, j} = 1, 
          && \forall j \in \mathcal{N},\\
      & \sum_{i \in \mathcal{S}, j \in \mathcal{S}} x_{i, j} \leq |\mathcal{S}| - 1, 
          && \forall \mathcal{S} \subseteq \mathcal{N}, 2 \leq |\mathcal{S}| \leq N-1\\
      & x_{i, j} \in \{0, 1\}, 
          && \forall i, j \in \mathcal{N}^{2}
\end{align}

where
* $x_{i, j} = 1$ if city $j$ follows city $i$ in the tour, and zero otherwise
* $D_{i, j}$ is the (euclidean) distance between cities $i$ and $j$
* Constraints $\sum_{i \in \mathcal{S}, j \in \mathcal{S}} x_{i, j} \leq |\mathcal{S}| - 1$ are _subtour elimination constraints_

⚠️ There are exponentially many subtours! ⚠️

In [3]:
function generate_tsp_data(N; seed=42)
    Random.seed!(seed)
    
    # Sample coordinates in the [0, 1] \times [0, 1] square
    X = rand(N, 2)
    
    # Compute Euclidean distances
    D = zeros(N, N)
    for i in 1:N
        xᵢ = X[i, :]
        for j in (i+1):N
            xⱼ = X[j, :]
            # Euclidean distances
            D[i, j] = D[j, i] = norm(xᵢ - xⱼ, 2)
        end
    end
    
    return X, D
end

generate_tsp_data (generic function with 1 method)

## Lazy constraints

In [76]:
function extract_tour(x)
    n = size(x, 1)
    size(x, 1) == size(x, 2) || error("x must be square")
    
    # `t[i]` is the city that follows `i` in the tour
    t = zeros(Int, n)
    @inbounds for j in 1:n
        for i in 1:n
            x[i, j] == 1.0 && (t[i] = j; break;)
        end
    end
    
    flags = falses(n)
    for i in t
        flags[i] = true
    end
    all(flags) || error("City $(argmin(flags)) is not visited")
    
    return t
end

"""
    find_subtours(x)
Compute sub-tours from tentative tour `x`.
"""
function find_subtours(t)
    n = length(t)
    # List of all sub-tours in `t`
    # Will be empty if `t` is a tour
    subtours = Vector{Int}[]

    flags = falses(n)
    num_seen = 0
    while num_seen < n
        first = argmin(flags)  # first city in current tour
        flags[first] && break  # all cities have been visited
        subtour = [first]
        flags[first] = true
        num_seen += 1
        next = t[first]
        next == first && error("City $(first) is its self-successor.")
        while next != first
            push!(subtour, next)
            flags[next] = 1
            num_seen += 1
            next = t[next]
        end

        length(subtour) == n && (return [subtour])  # x is a tour
        # @info "Found sub-tour: $subtour"
        push!(subtours, subtour)
    end
    
#     @info "Found $(length(subtours)) sub-tours."
    return subtours
end

find_subtours

In [160]:
function solve_tsp_iterative(N; seed=42)
    t_start = time()
    
    # Generate TSP data
    X, D = generate_tsp_data(N; seed)
    
    # Build TSP model
    tsp = direct_model(Gurobi.Optimizer())

    @variable(tsp, x[1:N, 1:N], Bin)

    @constraint(tsp, no_self[i in 1:N], x[i, i] == 0)
    @constraint(tsp, arcs_in[j in 1:N], sum(x[:, j]) == 1)
    @constraint(tsp, arcs_out[i in 1:N], sum(x[i, :]) == 1)

    @objective(tsp, Min, sum(D .* x))

    # Iterative constraint generation
    solved = false
    niter = 0

    tours = []
    
    println("Solving a TSP with $(N) cities\n")
    
    @printf "%4s  %4s  %6s\n" "Iter" "#ST" "Time"
    while !solved && (niter < 30)
        niter += 1

        set_silent(tsp)
        optimize!(tsp)
        x_ = round.(value.(x));

        tour = extract_tour(x_)
        push!(tours, tour)
        subtours = find_subtours(tour)

        t_elapsed = time() - t_start
        @printf "%4d  %4d  %6.2f\n" niter length(subtours) t_elapsed

        if length(subtours) == 1
            solved = true
            println("TSP Solved at iteration $niter")
            break
        end

        # add cuts
        for st in subtours
            S = length(st)
            @constraint(tsp, 
                sum(x[st[1 + (k % S)], st[1 + ((k+1)%S)]] for k in 1:S) <= S-1
            )
            @constraint(tsp, 
                sum(x[st[1 + ((k+1)%S)], st[1 + (k % S)]] for k in 1:S) <= S-1
            )
        end
    end
    
    return X, D, tsp, tours
end

function plot_tour(X, tour)
    plt = scatter(
        legend=false,
        X[:, 1],
        X[:, 2]
    )
    
    subtours = find_subtours(tour)

    for st in subtours
        S = length(st)
        for k in 1:S
            iₖ = st[1 + (k % S)]
            jₖ = st[1 + ((k+1)%S)]
            plot!(plt,
                [X[iₖ, 1], X[jₖ, 1]],
                [X[iₖ, 2], X[jₖ, 2]],
                label=nothing,
                color=:black,
            )
        end
    end
    return plt
end

plot_tour (generic function with 1 method)

## Using a lazy constraint callback

In [176]:
function solve_tsp_lazy(N; seed=42)
    t_start = time()
    
    # Generate TSP data
    X, D = generate_tsp_data(N; seed)
    
    # Build TSP model
    tsp = direct_model(Gurobi.Optimizer())

    @variable(tsp, x[1:N, 1:N], Bin)

    @constraint(tsp, no_self[i in 1:N], x[i, i] == 0)
    @constraint(tsp, arcs_in[j in 1:N], sum(x[:, j]) == 1)
    @constraint(tsp, arcs_out[i in 1:N], sum(x[i, :]) == 1)

    @objective(tsp, Min, sum(D .* x))
    
    tours = []
    
    # Define lazy callback
    function tsp_callback_lazy(cb_data)
        # Only separate cuts at integer solutions
        cb_status = callback_node_status(cb_data, tsp)
        cb_status == MOI.CALLBACK_NODE_STATUS_INTEGER || return nothing
        
        @info typeof(cb_data) maxlog=10
        
        # Grab solution and extract subtours
        x_ = round.(callback_value.(cb_data, x))
        tour = extract_tour(x_)
        push!(tours, tour)
        subtours = find_subtours(tour)
        
        length(subtours) == 1 && return nothing  # Solution is a tour
        
        # Add lazy constraints
        for st in subtours
            S = length(st)
            con1 = @build_constraint(
                sum(x[st[1 + (k % S)], st[1 + ((k+1)%S)]] for k in 1:S) <= S-1
            )
            con2 = @build_constraint(
                sum(x[st[1 + ((k+1)%S)], st[1 + (k % S)]] for k in 1:S) <= S-1
            )
            
            MOI.submit(tsp, MOI.LazyConstraint(cb_data), con1)
            MOI.submit(tsp, MOI.LazyConstraint(cb_data), con2)
        end
        
        return nothing
    end
    
    MOI.set(tsp, MOI.LazyConstraintCallback(), tsp_callback_lazy)
    
    println("Solving a TSP with $(N) cities\n")
    
    unset_silent(tsp)
    optimize!(tsp)
    
    return X, D, tsp, tours
end

solve_tsp_lazy (generic function with 1 method)

In [177]:
N = 50
seed = 42

42

In [166]:
X, D, tsp, tours = solve_tsp_iterative(N; seed);

@show objective_value(tsp)

ntours = length(tours)
anim = @animate for t in 1:ntours
    plt = plot_tour(X, tours[t])
    title!(plt, "TSP solution at iteration $(t)")
    plt
end

gif(anim, "tsp_iterative_$(N).gif", fps=3);

Academic license - for non-commercial use only - expires 2022-07-04
Solving a TSP with 30 cities

Iter   #ST    Time
   1    12    0.01
   2     5    0.04
   3     3    0.09
   4     2    0.15
   5     3    0.26
   6     1    0.35
TSP Solved at iteration 6
objective_value(tsp) = 4.634958049385056


┌ Info: Saved animation to 
│   fn = /mnt/c/Users/mathi/Git/or_tutorials/callbacks/tsp_iterative_30.gif
└ @ Plots /home/mtanneau/.julia/packages/Plots/tXtrW/src/animation.jl:114


In [178]:
X, D, tsp, tours = solve_tsp_lazy(N; seed);

# ntours = length(tours)
# anim = @animate for t in 1:ntours
#     plt = plot_tour(X, tours[t])
#     title!(plt, "TSP solution at iteration $(t)")
#     plt
# end

# gif(anim, "tsp_lazy_$(N).gif", fps=6);

Academic license - for non-commercial use only - expires 2022-07-04
Solving a TSP with 50 cities

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 150 rows, 2500 columns and 5050 nonzeros
Model fingerprint: 0x8b041c78
Variable types: 0 continuous, 2500 integer (2500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 50 rows and 50 columns
Presolve time: 0.01s
Presolved: 100 rows, 2450 columns, 4900 nonzeros
Variable types: 0 continuous, 2450 integer (2450 binary)

Root relaxation: objective 5.173253e+00, 113 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.17325    0   16          -    5.17325      -     - 

┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26
┌ Info: Gurobi.CallbackData
└ @ Main In[176]:26


In [175]:
ntours = length(tours)
anim = @animate for t in 1:ntours
    plt = plot_tour(X, tours[t])
    title!(plt, "Solution at callback call $(t)")
    plt
end

gif(anim, "/home/mtanneau/Windows/tsp_lazy_$(N).gif", fps=24);

┌ Info: Saved animation to 
│   fn = /home/mtanneau/Windows/tsp_lazy_100.gif
└ @ Plots /home/mtanneau/.julia/packages/Plots/tXtrW/src/animation.jl:114


## Heuristics